In [40]:
import os
import PIL.Image
import numpy as np
import pickle

In [18]:
cluster_list = './curriculum_clustering/input/webvision_cls0-9.txt' # webvision train list
with open(cluster_list) as f:
    metadata = [x.strip().split(' ') for x in f]

file_set = set([item[0] for item in metadata])
file_list = [item[0] for item in metadata]
print(file_list[0])
#print(metadata[0])
print("# file_set:", len(file_set))

flickr/q0001/2680197397.jpg
# file_set: 25744


In [52]:
folder = "./flickr_resized_256/flickr/"
folders = [x[0] for x in os.walk(folder)]
folders = folders[1:]
#print(folders)

X = []
img_ids = []
for fld in folders:
    #print(fld)
    for file in os.listdir(fld):
        img_id = fld +"/"+ file
        img_id = "/".join(img_id.split('/')[-3:])
        file = os.path.join(fld, file)
        #print(img_id)
        #break
        if img_id in file_set:
            #print(img_id)
            img = PIL.Image.open(file)
            img = np.array(img)
            X.append(img)
            img_ids.append(img_id)
        
print("# X:", len(X))

# X: 14502


In [45]:
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [42]:
def repickle(res, file):
    pickle.dump(res, open(file, "wb" ))

In [55]:
data = dict()
data['images'] = X[:7251]
data['id'] = img_ids[:7251]
repickle(data, './curriculum_clustering/input/webvision_cls0-9_batch1')

data = dict()
data['images'] = X[7251:]
data['id'] = img_ids[7251:]
repickle(data, './curriculum_clustering/input/webvision_cls0-9_batch2')

In [46]:
X_p = unpickle('./curriculum_clustering/input/webvision_cls0-9_data')

In [47]:
print(len(X_p))
print(X_p[0])

2853
[[[ 43  70  61]
  [ 45  72  63]
  [ 51  79  67]
  ...
  [ 86 116 104]
  [ 96 124 110]
  [126 153 138]]

 [[ 34  64  56]
  [ 38  68  60]
  [ 43  73  63]
  ...
  [ 70  98  86]
  [134 160 147]
  [208 235 220]]

 [[ 24  53  48]
  [ 27  56  51]
  [ 30  61  55]
  ...
  [ 72 100  88]
  [165 191 178]
  [237 255 247]]

 ...

 [[ 56  96  85]
  [ 54  94  83]
  [ 52  92  81]
  ...
  [ 17  37  36]
  [ 16  38  36]
  [ 16  38  36]]

 [[ 59 101  89]
  [ 55  97  85]
  [ 49  93  80]
  ...
  [ 17  37  36]
  [ 16  38  36]
  [ 16  38  36]]

 [[ 58 101  91]
  [ 54  97  87]
  [ 47  93  82]
  ...
  [ 16  36  35]
  [ 15  37  35]
  [ 14  36  34]]]
